In [1]:
from root_system import RootSystem, WeightSpaceElement
import itertools
from functools import cache
from operator import methodcaller, attrgetter, neg
from collections import defaultdict
from typing import Set, List, Dict
from numbers import Number
from random import shuffle

In [2]:
type EquationTerms = List[tuple[WeightSpaceElement, WeightSpaceElement]]


def is_adjoint_weight(root_system: RootSystem, weight: WeightSpaceElement) -> bool:
    return weight in root_system or not weight


def equation_terms(root_system: RootSystem, weight: WeightSpaceElement) -> EquationTerms:
    """
    Equation on the orbit of the highest weight vector of the adjoint representation
    corresponding to a pair of non-zero weights (pi, weight).
    """
    if weight not in root_system:
        raise ValueError('Weight parameter incorrect: only roots allowed.')
    pi = root_system.maximal_root
    weight_pairs = [(pi, weight)]
    weight_pairs.extend((pi - a, weight + a) for a in root_system.positive_roots() if is_adjoint_weight(root_system, pi - a) and is_adjoint_weight(root_system, weight + a))
    return weight_pairs


def equation_terms_reduced(root_system: RootSystem, weight: WeightSpaceElement, vanished_weights: Set[WeightSpaceElement]) -> EquationTerms:
    """
    Equations on the orbit of the highest weight vector with term appearing among the vanished weights deleted.
    """
    eq_terms = equation_terms(root_system, weight)
    return [(w1, w2) for w1, w2 in eq_terms if w1 not in vanished_weights and w2 not in vanished_weights]

In [3]:
def deduce_vanishing(root_system: RootSystem, vanished_weights: Set[WeightSpaceElement], verbose: bool = False) -> None:
    """
    Updates the list of weights which vanish according to the equations.
    """
    pi = root_system.maximal_root
    new_vanished_weights = True
    iteration = 0
    while new_vanished_weights:
        new_vanished_weights = set()
        iteration += 1
        for weight in root_system.all_roots():
            if weight != pi and weight not in vanished_weights and any(weight):
                if len(equation_terms_reduced(root_system, weight, vanished_weights)) == 1:
                    new_vanished_weights.add(weight)
        vanished_weights.update(new_vanished_weights)
        if verbose and new_vanished_weights:
            print(' Iteration on vanishing deduction: {}'.format(iteration))
            for weight in new_vanished_weights:
                print('  v[{}]=0'.format(weight))

In [4]:
def beta_series(beta: WeightSpaceElement, weight: WeightSpaceElement, root_system: RootSystem) -> Set[WeightSpaceElement]:
    """
    Beta-series through weight, including zero weight.
    """
    series = set()
    for sign in (1, -1):
        signed_beta = sign * beta
        shifted_weight = weight
        while is_adjoint_weight(root_system, shifted_weight):
            series.add(shifted_weight)
            shifted_weight += signed_beta
    return series


def all_beta_series(beta: WeightSpaceElement, root_system: RootSystem):
    """
    All beta-series.
    """
    series = set()
    for weight in root_system.all_roots():
        series_through_weight = beta_series(beta, weight, root_system)
        if len(series_through_weight) > 1:
            series.add(frozenset(series_through_weight))
    return series


def beta_series_r(beta: WeightSpaceElement, weight: WeightSpaceElement, root_system: RootSystem) -> int:
    """
    Left end of the beta-series through weight. 
    """
    r = 0
    shifted_weight = weight
    while is_adjoint_weight(root_system, shifted_weight):
        shifted_weight -= beta
        r += 1
    return r - 1


def beta_series_q(beta: WeightSpaceElement, weight: WeightSpaceElement, root_system: RootSystem) -> int:
    """
    Right end of the beta-series through weight. 
    """
    q = 0
    shifted_weight = weight
    while is_adjoint_weight(root_system, shifted_weight):
        shifted_weight += beta
        q += 1
    return q - 1

In [5]:
def int_div(a, b):
    q, r = divmod(a, b)
    if r == 0:
        return q
    else:
        raise ValueError('Integer division by a non-divisor.')


def equation_coefficients_reduced(root_system: RootSystem, weight: WeightSpaceElement, vanished_weights: Set[WeightSpaceElement]) -> List[tuple[Number, tuple[WeightSpaceElement, WeightSpaceElement]]]:
    """
    Reduced equation terms together with the coefficients. 
    """
    eq_terms = equation_terms_reduced(root_system, weight, vanished_weights)
    zero = root_system.zero_weight
    if any(zero in term for term in eq_terms):
        raise ValueError('Coefficients for equations with zero weights not supported.')
    pi = root_system.maximal_root
    coefficients = []
    for weight1, weight2 in eq_terms:
        if weight1 == pi:
            coefficients.append(root_system.scalar_product(pi, weight - pi))
        else:
            alpha = pi - weight1
            alpha_norm = root_system.scalar_product(alpha, alpha)
            r_alpha_pi = beta_series_r(alpha, pi, root_system)
            q_alpha_beta = beta_series_q(alpha, weight, root_system)
            coefficients.append(int_div(alpha_norm * r_alpha_pi * q_alpha_beta, 2))
    return list(zip(coefficients, eq_terms))

In [6]:
def reduction_step_allowed(root_system: RootSystem, beta: WeightSpaceElement, vanished_weights: Set[WeightSpaceElement], verbose: bool = False) -> bool:
    """
    Checks whether the reduction using the root `beta` is allowed.
    """
    beta_series = all_beta_series(beta, root_system)
    pi = root_system.maximal_root
    no_vanishing_obstacles = True
    no_sl2_equations_obstacles = True
    for bs in beta_series:
        if pi not in bs:
            if not bs.isdisjoint(vanished_weights) and not bs.issubset(vanished_weights):
                no_vanishing_obstacles = False
                break
        elif len(bs) > 2:
            if root_system.zero_weight not in bs:
                no_sl2_equations_obstacles = True
                for gamma in bs:
                    if gamma != pi:
                        cs = equation_coefficients_reduced(root_system, gamma, vanished_weights)
                        if verbose:
                            print(' Eq. [pi,{}]: {} = 0.'.format(gamma, ' ± '.join('{}*v[{}]v[{}]'.format(abs(c), *t) for c, t in cs)))
                        if len(cs) != 2 or abs(cs[0][0]) != abs(cs[1][0]):
                            no_sl2_equations_obstacles = False
                            break
            else:
                bs_lowest_weight = min(bs, key=attrgetter('ht'))
                eq_terms_reduced = equation_terms_reduced(root_system, bs_lowest_weight, vanished_weights)
                eq_weights = set(itertools.chain.from_iterable(eq_terms_reduced))
                no_sl2_equations_obstacles = bs == eq_weights
    return no_vanishing_obstacles and no_sl2_equations_obstacles

In [7]:
class ReductionException(BaseException):
    pass


def reduction(root_system: RootSystem, verbose: bool = False):
    if root_system.series in ('A', 'C', 'G'):
        # remove the next line in order to try for G2, but the code does not prove it completely
        raise ReductionException('Reduction for root systems of type {} not supported.'.format(root_system.series))
    if verbose:
        print('REDUCTION FOR {}{}'.format(root_system.series, root_system.rank))
    pi = root_system.maximal_root
    sigma = [a for a in root_system.positive_roots() if pi - a in root_system]
    sigma1 = [a for a in sigma if 2 * a.ht < pi.ht]
    sigma1.sort(key=attrgetter('ht'))
    sigma2 = [pi - a for a in reversed(sigma1)]
    # weights where the entries have been shown to be zero provided v[pi] != 0
    vanished_weights = set()
    # alternative vanishing in case v[pi] = 0
    alt_vanished_weights = {pi}
    for step, beta in enumerate(itertools.chain(sigma1, [pi], sigma2), start=1):
        if verbose:
            print('{}) Applying beta = {}...'.format(step, beta))
        if not reduction_step_allowed(root_system, beta, vanished_weights, verbose):
            raise ReductionException('Reduction step with beta = {} not allowed by the equations.'.format(beta))
        if verbose:
            print(' No obstacles for beta = {} found.'.format(beta))
        new_vanished_weights = beta_series(beta, pi, root_system)
        new_vanished_weights.discard(pi)
        vanished_weights.update(new_vanished_weights)
        if verbose:
            plural = len(new_vanished_weights) > 1
            print(' Entr{} made zero at weight{} {}.'.format(
                'ies' if plural else 'y', 's' if plural else '',
                ', '.join(map(str, sorted(new_vanished_weights, key=attrgetter('ht'))))
            ))
        deduce_vanishing(root_system, vanished_weights, verbose)
        # alternative vanishing
        weight = pi - beta
        print(' Alternative vanishing assuming v[{}]=0:'.format(weight))
        alt_vanished_weights.add(weight)
        for gamma in itertools.islice(sigma1, None, step):
            source = weight - gamma
            if is_adjoint_weight(root_system, source):
                if source not in alt_vanished_weights:
                    alt_vanished_weights.add(source)
                    print('  v[{}]=0'.format(source))
                    if source == root_system.zero_weight:
                        alt_vanished_weights.add(source - gamma)
                        print('  v[{}]=0'.format(source - gamma))
    adjoint_weights = set(itertools.chain(root_system.all_roots(), [root_system.zero_weight]))
    non_vanished_weights = adjoint_weights.difference(vanished_weights)
    return non_vanished_weights == {pi} and alt_vanished_weights.issuperset(root_system.positive_roots())

In [8]:
reductions_results = {}
for series, rank in [('D', 4), ('B', 3), ('E', 6), ('E', 7), ('E', 8), ('F', 4)]:
    rs = RootSystem(series, rank)
    rs_name = '{}{}'.format(series, rank)
    reductions_results[rs_name] = reduction(rs, verbose=True)
    print()
print('RESULTS:')
for rs_name, result in reductions_results.items():
    print(' {}: {}'.format(rs_name, 'success' if result else 'fail'))

REDUCTION FOR D4
1) Applying beta = (0,1,0,0)...
 No obstacles for beta = (0,1,0,0) found.
 Entry made zero at weight (1,1,1,1).
 Alternative vanishing assuming v[(1,1,1,1)]=0:
2) Applying beta = (0,1,0,1)...
 No obstacles for beta = (0,1,0,1) found.
 Entry made zero at weight (1,1,1,0).
 Iteration on vanishing deduction: 1
  v[(0,0,1,0)]=0
  v[(1,0,0,0)]=0
 Iteration on vanishing deduction: 2
  v[(0,-1,0,-1)]=0
  v[(0,-1,0,0)]=0
 Alternative vanishing assuming v[(1,1,1,0)]=0:
3) Applying beta = (1,1,0,0)...
 No obstacles for beta = (1,1,0,0) found.
 Entry made zero at weight (0,1,1,1).
 Iteration on vanishing deduction: 1
  v[(0,0,0,1)]=0
 Iteration on vanishing deduction: 2
  v[(-1,-1,0,0)]=0
 Alternative vanishing assuming v[(0,1,1,1)]=0:
  v[(0,0,1,0)]=0
4) Applying beta = (0,1,1,0)...
 No obstacles for beta = (0,1,1,0) found.
 Entry made zero at weight (1,1,0,1).
 Iteration on vanishing deduction: 1
  v[(0,-1,-1,0)]=0
 Alternative vanishing assuming v[(1,1,0,1)]=0:
  v[(1,0,0,0)]=